Sign in sheet:

Sabeeh: OUT

Ryan: OUT

# Set up

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Libraries

In [ ]:
import os
import imageio
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
import random
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import torchvision.models as models

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np

import keras
from keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications import InceptionV3

%cd /content/drive/MyDrive/Bio Final Project

Organize Data:
- Append it into a dictionary where the keys are the filename and values are the label

In [ ]:
data = {}
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      data[filename] = 'mild'

In [ ]:
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      data[filename] = 'moderate'

In [ ]:
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      data[filename] = 'non'

In [ ]:
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      data[filename] = 'verymild'

Need to use integers not strings

In [ ]:

for item in data:
  if data[item] == 'mild':
    data[item] = 0
  if data[item] == 'verymild':
    data[item] = 1
  if data[item] == 'moderate':
    data[item] = 2
  if data[item] == 'non':
    data[item] = 3

# RESNET

In [ ]:
# RUN
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        original = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(original)
        out = F.relu(out)
        
        return out

In [ ]:
#RUN
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64      
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)    
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out) 

        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
                    
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
#DO NOT RUN, TRAINED MODEL DOES NOT NEED TO BE RUN AGAIN
num_epochs = 5
running_loss = 0
class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        if label == 0:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
        elif label == 2:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
        elif label == 1:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
        else:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([    transforms.Resize(32),    transforms.ToTensor(),    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(data, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18()
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

i = 0
for epoch in range(num_epochs):
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        i += 1

In [ ]:
#DO NOT RUN
model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/resnet.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
#DO NOT RUN UNLESS YOU WANT TO EVALUATE THE MODEL
#This code is provided in the "Load Models" section for later use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNetModel = ResNet18()
ResNetModel = ResNetModel.to(device)

model_path = '/content/drive/MyDrive/Bio Final Project/Models/resnet.pth'
ResNetModel.load_state_dict(torch.load(model_path))


In [ ]:
#DO NOT RUN
test_dataset = MyDataset(data, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

ResNetModel.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        # Move data to GPU if available
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        # Forward pass
        outputs = ResNetModel(images)

        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # Calculate accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on test set: {:.2f}%'.format(100 * accuracy))

# Accuracy on test set: 99.44%

# VGG16

In [ ]:
#DO NOT RUN, TRAINED MODEL DOES NOT NEED TO BE RUN AGAIN
num_epochs = 5
running_loss = 0
class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        if label == 0:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
        elif label == 2:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
        elif label == 1:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
        else:
          os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image, label

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(data, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


import torchvision.models as models

model = models.vgg16(pretrained=True)

num_classes = 4
model.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, betas=(0.9, 0.999))

i=0
for epoch in range(num_epochs):
    for images, labels in dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        i += 1

In [ ]:
#DO NOT RUN
model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/vgg.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
#DO NOT RUN UNLESS YOU WANT TO EVALUATE THE MODEL
#This code is provided in the "Load Models" section for later use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vggmodel = models.vgg16(pretrained=False) 
num_classes = 4
vggmodel.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

vggmodel = vggmodel.to(device)

model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/vgg.pth'
vggmodel.load_state_dict(torch.load(model_save_path))

In [ ]:
#DO NOT RUN
test_dataset = MyDataset(data, transform=preprocess)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

vggmodel.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        outputs = vggmodel(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on test set: {:.2f}%'.format(100 * accuracy))

# Accuracy on test set: 46.18%

# Inception-V3

In [ ]:
#DO NOT RUN, TRAINED MODEL DOES NOT NEED TO BE RUN AGAIN
num_epochs = 5
running_loss = 0

class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        if label == 0:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
        elif label == 2:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
        elif label == 1:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
        else:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize(299),   
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(data, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = models.inception_v3(pretrained=True)

num_classes = 4
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

i = 0
for epoch in range(num_epochs):
    for images, labels in dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs, _ = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        i += 1

In [ ]:
#DO NOT RUN
model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/inception.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
#DO NOT RUN UNLESS YOU WANT TO EVALUATE THE MODEL
#This code is provided in the "Load Models" section for later use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inceptionmodel = models.inception_v3(pretrained=False)
num_classes = 4
inceptionmodel.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)

inceptionmodel = inceptionmodel.to(device)

model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/inception.pth'
inceptionmodel.load_state_dict(torch.load(model_save_path))

In [ ]:
#DO NOT RUN
test_dataset = MyDataset(data, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

inceptionmodel.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        outputs = inceptionmodel(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on test set: {:.2f}%'.format(100 * accuracy))

# Accuracy on test set: 90.48%

# AlexNet (taken from GitHub)

In [ ]:
#RUN
!apt-get install git -y

In [ ]:
#RUN
!git clone https://github.com/wangyirui/AD_Prediction.git

In [ ]:
#RUN
%cd ./AD_Prediction/

In [ ]:
#RUN
from AlexNet2D import alexnet

In [ ]:
#DO NOT RUN, TRAINED MODEL DOES NOT NEED TO BE RUN AGAIN
num_epochs = 5
running_loss = 0

class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        if label == 0:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
        elif label == 2:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
        elif label == 1:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
        else:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
      
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(data, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = alexnet(pretrained=True)

num_classes = 4
num_features = model.classifier[6].in_features
model.classifier.fc_out = nn.Linear(1000, num_classes)

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))


for epoch in range(num_epochs):
    i=0
    for images, labels in dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        i += 1


In [ ]:
#DO NOT RUN
model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/alexnet.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
#DO NOT RUN UNLESS YOU WANT TO EVALUATE THE MODEL
#This code is provided in the "Load Models" section for later use
def load_trained_model(model_path):
    model = alexnet(pretrained=True)

    num_classes = 4
    num_features = model.classifier[6].in_features
    model.classifier.fc_out = nn.Linear(1000, num_classes)

    model.load_state_dict(torch.load(model_path))

    if torch.cuda.is_available():
        model = model.cuda()

    return model

model_load_path = '/content/drive/MyDrive/Bio Final Project/Models/alexnet.pth'
ANmodel = load_trained_model(model_load_path)

In [ ]:
#DO NOT RUN
num_epochs = 5
running_loss = 0

class MyDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        if label == 0:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
        elif label == 2:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
        elif label == 1:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
        else:
            os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
      
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = MyDataset(data, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
test_dataset = MyDataset(data, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

ANmodel.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        outputs = ANmodel(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on test set: {:.2f}%'.format(100 * accuracy))

# Accuracy on test set: 89.51%

# Load Models

In [ ]:
#Load ResNet.  Need to run the first 2 blocks of ResNet section before this
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNetModel = ResNet18()
ResNetModel = ResNetModel.to(device)

model_path = '/content/drive/MyDrive/Bio Final Project/Models/resnet.pth'
ResNetModel.load_state_dict(torch.load(model_path))

#Load VGG-16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vggmodel = models.vgg16(pretrained=False) 
num_classes = 4
vggmodel.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

vggmodel = vggmodel.to(device)

model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/vgg.pth'
vggmodel.load_state_dict(torch.load(model_save_path))

#Load Inception-V3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inceptionmodel = models.inception_v3(pretrained=False)
num_classes = 4
inceptionmodel.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)

inceptionmodel = inceptionmodel.to(device)

model_save_path = '/content/drive/MyDrive/Bio Final Project/Models/inception.pth'
inceptionmodel.load_state_dict(torch.load(model_save_path))

#Load Alex-Net.  Need to run the first 4 sections of Alexnet section
def load_trained_model(model_path):
    model = alexnet(pretrained=True)

    num_classes = 4
    num_features = model.classifier[6].in_features
    model.classifier.fc_out = nn.Linear(1000, num_classes)

    model.load_state_dict(torch.load(model_path))

    if torch.cuda.is_available():
        model = model.cuda()

    return model

model_load_path = '/content/drive/MyDrive/Bio Final Project/Models/alexnet.pth'
ANmodel = load_trained_model(model_load_path)


# Grad Cam

In [ ]:
#RUN
import cv2
import numpy as np
import torch
class GradCamRes:
    def __init__(self, model, device):
        self.model = model
        self.model.eval()
        self.device = device
        self.gradient = None
        self.activations = None
        self.hooks = []
        self.hooks.append(self.model.layer4[1].conv2.register_backward_hook(self.save_gradient))
        self.hooks.append(self.model.layer4[1].conv2.register_forward_hook(self.save_activation))

    def save_gradient(self, module, grad_input, grad_output):
        self.gradient = grad_output[0].to(self.device)

    def save_activation(self, module, input, output):
        self.activations = output.to(self.device)

    def forward(self, input):
        return self.model(input)

    def backward_on_target(self, output, target):
        self.model.zero_grad()
        one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().to(self.device)
        one_hot_output[0][target] = 1
        output.backward(gradient=one_hot_output, retain_graph=True)

    def generate_cam(self, input_tensor, target_class=None):
        output = self.forward(input_tensor)
        if target_class is None:
            target_class = output.max(1)[-1].item()
        self.backward_on_target(output, target_class)
        gradient = self.gradient.data.cpu().numpy()[0]
        activations = self.activations.data.cpu().numpy()[0]
        weights = np.mean(gradient, axis=(1, 2))
        cam = np.sum((weights * activations.T), axis=2).T
        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input_tensor.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam

In [ ]:
#RUN
class GradCamVGG:
    def __init__(self, model, device):
        self.model = model
        self.model.eval()
        self.device = device
        self.gradient = None
        self.activations = None
        self.hooks = []
        self.hooks.append(self.model.features[28].register_backward_hook(self.save_gradient))
        self.hooks.append(self.model.features[28].register_forward_hook(self.save_activation))

    def save_gradient(self, module, grad_input, grad_output):
        self.gradient = grad_output[0].to(self.device)

    def save_activation(self, module, input, output):
        self.activations = output.to(self.device)

    def forward(self, input):
        return self.model(input)

    def backward_on_target(self, output, target):
        self.model.zero_grad()
        one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().to(self.device)
        one_hot_output[0][target] = 1
        output.backward(gradient=one_hot_output, retain_graph=True)

    def generate_cam(self, input_tensor, target_class=None):
        output = self.forward(input_tensor)
        if target_class is None:
            target_class = output.max(1)[-1].item()
        self.backward_on_target(output, target_class)
        gradient = self.gradient.data.cpu().numpy()[0]
        activations = self.activations.data.cpu().numpy()[0]
        weights = np.mean(gradient, axis=(1, 2))  
        cam = np.sum((weights * activations.T), axis=2).T
        cam = np.maximum(cam, 0) 
        cam = cv2.resize(cam, input_tensor.shape[2:])  
        cam = cam - np.min(cam) 
        cam = cam / np.max(cam)
        return cam

In [ ]:
#RUN
class GradCamInceptionV3:
    def __init__(self, model, device):
        self.model = model.to(device)
        self.model.eval()
        self.last_conv_layer = model.Mixed_7c  
        self.gradient = None
        self.activations = None
        self.device = device

        self.hooks = []
        self.hooks.append(self.last_conv_layer.register_forward_hook(self.save_activation))
        self.hooks.append(self.last_conv_layer.register_backward_hook(self.save_gradient))

    def save_gradient(self, module, grad_input, grad_output):
        self.gradient = grad_output[0]

    def save_activation(self, module, input, output):
        self.activations = output

    def forward(self, input):
        return self.model(input)

    def backward_on_target(self, output, target):
        self.model.zero_grad()
        one_hot_output = torch.zeros(1, output.size()[-1]).to(self.device)
        one_hot_output[0][target] = 1
        output.backward(gradient=one_hot_output, retain_graph=True)

    def generate_cam(self, input_tensor, target_class=None):
        input_tensor = input_tensor.to(self.device)
        output = self.forward(input_tensor)
        if target_class is None:
            target_class = output.max(1)[-1].item()
        self.backward_on_target(output, target_class)

        if self.gradient is None or self.activations is None:
            raise RuntimeError("Gradient or activations are missing. Make sure hooks are properly registered.")

        gradients = self.gradient
        activations = self.activations
        weights = torch.mean(gradients, dim=(2, 3))  
        cam = torch.sum(weights[..., None, None] * activations, dim=1)
        cam = torch.relu(cam)
        cam = F.interpolate(cam.unsqueeze(0), size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = cam.squeeze().detach().cpu().numpy()
        cam = np.maximum(cam, 0) 

        max_value = np.max(cam)
        if max_value != 0:
            cam = cam / max_value  

        return cam

In [ ]:
#RUN
import torch.nn as nn
from torch.autograd import Function
from torchvision.models.resnet import ResNet

class ANGradCam:
    def __init__(self, model, target_layer_index):
        self.model = model
        self.target_layer = self.model.features[target_layer_index]
        self.gradient = None
        self.conv_output = None

        self.target_layer.register_backward_hook(self.save_gradient)
        self.target_layer.register_forward_hook(self.save_output)

    def save_gradient(self, module, grad_input, grad_output):
        self.gradient = grad_output[0]

    def save_output(self, module, input, output):
        self.conv_output = output

    def forward_pass(self, input_image):
        return self.model(input_image)

    def backward_pass(self, output, class_index):
        one_hot = torch.zeros_like(output)
        one_hot[0][class_index] = 1
        output.backward(gradient=one_hot, retain_graph=True)

    def generate_heatmap(self, input_image, class_index):
        self.model.zero_grad()

        output = self.forward_pass(input_image)

        self.backward_pass(output, class_index)

        gradient = self.gradient

        alpha = gradient.mean(dim=[2, 3], keepdim=True)

        weighted_activations = (alpha * self.conv_output).sum(dim=1, keepdim=True)

        heatmap = nn.ReLU()(weighted_activations)
        heatmap = nn.functional.interpolate(heatmap, size=input_image.shape[2:], mode='bilinear', align_corners=False)

        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

        return heatmap.squeeze().cpu().detach().numpy()

In [ ]:
#RUN
import matplotlib.pyplot as plt
import cv2

def getHeatmap(label, model, name):
  if label == 0:
    directory_path = '/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented'
  if label == 1:
    directory_path = '/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented'
  if label == 2:
   directory_path = '/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented'
  if label == 3:
    directory_path = '/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented'

  
  files = os.listdir(directory_path)
  files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]
  random_file = random.choice(files)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  if name == "Alex":
    grad_cam = ANGradCam(model, 10)
    image_path = os.path.join(directory_path, random_file)

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    class_index = data[random_file] 
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    input_batch = input_batch.to(device)
    heatmap = grad_cam.generate_heatmap(input_batch, class_index)

  if name == "ResNet":
    preprocess = transforms.Compose([    transforms.Resize(32),    transforms.ToTensor(),    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_path = os.path.join(directory_path, random_file)
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_tensor = input_tensor.to(device)

    grad_cam = GradCamRes(model, device)
    heatmap = grad_cam.generate_cam(input_tensor, target_class=label)

  if name == "VGG":
    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_path = os.path.join(directory_path, random_file)
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_tensor = input_tensor.to(device)

    grad_cam = GradCamVGG(model, device)
    heatmap = grad_cam.generate_cam(input_tensor, target_class=label)

  if name == "Inception":
    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_path = os.path.join(directory_path, random_file)
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image).unsqueeze(0).to(device)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_tensor = input_tensor.to(device)

    grad_cam = GradCamInceptionV3(model, device)
    heatmap = grad_cam.generate_cam(input_tensor, target_class=label)

  original_image = cv2.imread(image_path)
  original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

  heatmap_normalized = cv2.normalize(heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

  heatmap_resized = cv2.resize(heatmap_normalized, (original_image.shape[1], original_image.shape[0]))

  heatmap_resized = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

  heatmap_resized_bgr = cv2.cvtColor(heatmap_resized, cv2.COLOR_RGB2BGR)

  superimposed_img = cv2.addWeighted(original_image, 0.7, heatmap_resized_bgr, 0.3, 0)

  plt.figure(figsize=(10, 4))

  plt.subplot(1, 2, 1)
  plt.imshow(original_image)
  plt.title('Original Image')

  plt.subplot(1, 2, 2)
  plt.imshow(superimposed_img)
  plt.title('Image with Heatmap')

  plt.show()

In [ ]:
#INSTRUCTIONS:
#FIRST SELECT A MODEL TO USE, THEN SELECT DEGREE OF DIMENTIA
#WILL GET AN ERROR IF YOU DO NOT SELECT A MODEL FIRST
#HEATMAPS LOOK DIFFERENT DEPENDING ON ACCURARY/TYPE OF MODEL

import ipywidgets as widgets
from IPython.display import display, clear_output

def button_click_event(button):
    label = button_to_label[button]
    with output:
        clear_output() 
        getHeatmap(label, model_used, model_name)

model_name = None
model_used = None

def res_click():
    global model_used, model_name
    label = 0
    model_name = "ResNet"
    model_used = ResNetModel
    model_name_widget.value = "Currently Using Model: " + model_name

def vgg_click():
    global model_used, model_name
    label = 1
    model_name = "VGG"
    model_used = vggmodel
    model_name_widget.value = "Currently Using Model: " + model_name

def inception_click():
    global model_used, model_name
    label = 2
    model_name = "Inception"
    model_used = inceptionmodel
    model_name_widget.value = "Currently Using Model: " + model_name

def alex_click():
    global model_used, model_name
    label = 3
    model_name = "Alex"
    model_used = ANmodel
    model_name_widget.value = "Currently Using Model: " + model_name

# Create the buttons
button1 = widgets.Button(description="Mild")
button2 = widgets.Button(description="Very Mild")
button3 = widgets.Button(description="Moderate")
button4 = widgets.Button(description="None")

output = widgets.Output()

# Create a dictionary to map buttons to labels
button_to_label = {
    button1: 0,
    button2: 1,
    button3: 2,
    button4: 3
}

# Attach the functions to the buttons' click events
button1.on_click(lambda _: button_click_event(button1))
button2.on_click(lambda _: button_click_event(button2))
button3.on_click(lambda _: button_click_event(button3))
button4.on_click(lambda _: button_click_event(button4))

modelbutton1 = widgets.Button(description="ResNet")
modelbutton2 = widgets.Button(description="VGG16")
modelbutton3 = widgets.Button(description="InceptionV3")
modelbutton4 = widgets.Button(description="AlexNet")

modelbutton1.on_click(lambda _: res_click())
modelbutton2.on_click(lambda _: vgg_click())
modelbutton3.on_click(lambda _: inception_click())
modelbutton4.on_click(lambda _: alex_click())

model_name_widget = widgets.Label(value="Currently Using Model: None")

# Create an HBox layout to horizontally align the buttons
buttons_layout2 = widgets.HBox([modelbutton1, modelbutton2, modelbutton3, modelbutton4])
buttons_layout = widgets.HBox([button1, button2, button3, button4])

# Display the buttons and model name
display(model_name_widget)
display(buttons_layout2)
display(buttons_layout, output)

# Fake Dimentia Image Generation

In [ ]:
#RUN
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

We don't actually have to run anything in the next 4 sections because we save all the results to a path.  In each of them is their training loops an example of what is generated

## Moderate

In [ ]:
#DO NOT RUN
mod = {}
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      mod[filename] = 'moderate'

In [ ]:
#DO NOT RUN, TRAINED MODEL DOESNT NEED TO BE RUN
import os
import zipfile 
import gdown
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

class CelebADataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Moderate_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image

image_size = 128
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

dataset = CelebADataset(mod, transform=transform)

batch_size = 1
num_workers = 0 if device.type == 'cuda' else 1
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=True)

In [ ]:
#DO NOT RUN
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)



In [ ]:
#DO NOT RUN
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
#DO NOT RUN
batch_size = 128
image_size = 128
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 8
lr = 0.0003
beta1 = 0.5
ngpu = 1
netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)
criterion = nn.BCELoss()
fixed_noise_mod = torch.randn(64, nz, 1, 1, device=device)
real_label = 1
fake_label = 0
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# DO NOT RUN, Training Loop

img_list = []
G_losses = []
D_losses = []
iters = 0

plt.figure(figsize=(10,10))
print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(celeba_dataloader, 0):
        
        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 0, device=device).type(torch.float32)                
        output = netD(real_cpu).view(-1)              
        label.fill_(real_label)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)        
        output = netD(fake.detach()).view(-1)              
        label.fill_(fake_label)        
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label) 
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i % 50 == 0:
            dsp_str = '[%d/%d][%d/%d]' % (epoch, num_epochs, i, len(celeba_dataloader))
            print('%s\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (dsp_str, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))       
            with torch.no_grad():
                fake = netG(fixed_noise_mod).detach().cpu()
            fake = vutils.make_grid(fake[:64], padding=2, normalize=True)
            plt.axis("off")
            plt.title("Generated fake Images")
            plt.imshow(np.transpose(vutils.make_grid(fake, padding=2, normalize=True),(1,2,0)))
            plt.pause(1)

        G_losses.append(errG.item())
        D_losses.append(errD.item())
                    
        iters += 1

In [ ]:
#DO NOT RUN
from torchvision.utils import save_image

with torch.no_grad():
    final_fake_mod = netG(fixed_noise_mod).detach().cpu()
final_fake_grid_mod = vutils.make_grid(final_fake_mod, padding=2, normalize=True)


In [ ]:
#RUN
if not os.path.exists('results'):
    os.makedirs('results')

tensor_save_path_mod = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakemoderate.pt'
#torch.save(final_fake_grid_mod, tensor_save_path_mod)
fakemod = torch.load(tensor_save_path_mod)

In [ ]:
#DO NOT RUN
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Final Generated Fake Images")
plt.imshow(np.transpose(fakemod, (1, 2, 0)))
plt.show()

## Non

In [ ]:
#DO NOT RUN
non = {}
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      non[filename] = 'non'

In [ ]:
#DO NOT RUN, MODEL ALREADY TRAINED
import os
import zipfile 
import gdown
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

class CelebADataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Non_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image

image_size = 64
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

dataset = CelebADataset(non, transform=transform)

batch_size = 1
num_workers = 0 if device.type == 'cuda' else 1

pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=True)

In [ ]:
#DO NOT RUN
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)



In [ ]:
#DO NOT RUN
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
#DO NOT RUN
batch_size = 128
image_size = 64
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 5
lr = 0.0005
beta1 = 0.5
ngpu = 1

netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

criterion = nn.BCELoss()

fixed_noise_non = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# DO NOT RUN, Training Loop

img_list = []
G_losses = []
D_losses = []
iters = 0

plt.figure(figsize=(10,10))
print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(celeba_dataloader, 0):

        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 0, device=device).type(torch.float32)                
        output = netD(real_cpu).view(-1)           
        label.fill_(real_label)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)        
        output = netD(fake.detach()).view(-1)            
        label.fill_(fake_label)        
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label)  
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i % 50 == 0:
            dsp_str = '[%d/%d][%d/%d]' % (epoch, num_epochs, i, len(celeba_dataloader))
            print('%s\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (dsp_str, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))       
            with torch.no_grad():
                fake = netG(fixed_noise_non).detach().cpu()
            fake = vutils.make_grid(fake[:64], padding=2, normalize=True)
            plt.axis("off")
            plt.title("Generated fake Images")
            plt.imshow(np.transpose(vutils.make_grid(fake, padding=2, normalize=True),(1,2,0)))
            plt.pause(1)

        G_losses.append(errG.item())
        D_losses.append(errD.item())
                    
        iters += 1

In [ ]:
#DO NOT RUN
from torchvision.utils import save_image

with torch.no_grad():
    final_fake_non = netG(fixed_noise_non).detach().cpu()
final_fake_grid_non = vutils.make_grid(final_fake_non, padding=2, normalize=True)



In [ ]:
#RUN
if not os.path.exists('results'):
    os.makedirs('results')

tensor_save_path_non = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakenon.pt'
#torch.save(final_fake_grid_non, tensor_save_path_non)
fakenon = torch.load(tensor_save_path_non)

In [ ]:
#DO NOT RUN
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Final Generated Fake Images")
plt.imshow(np.transpose(fakenon, (1, 2, 0)))
plt.show()

##Very Mild

In [ ]:
#DO NOT RUN
vmild = {}
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
for filename in os.listdir():
    if filename.endswith('.jpg'):
      vmild[filename] = 'vmild'

In [ ]:
#DO NOT RUN, model already trained
import os
import zipfile 
import gdown
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')


class CelebADataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image

image_size = 64
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

dataset = CelebADataset(vmild, transform=transform)


batch_size = 2
num_workers = 0 if device.type == 'cuda' else 1
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=True)

In [ ]:
#DO NOT RUN
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

batch_size = 128
image_size = 64
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 5
lr = 0.0002
beta1 = 0.5
ngpu = 1

netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

criterion = nn.BCELoss()

fixed_noise_vmild = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
#DO NOT RUN, Training Loop

img_list = []
G_losses = []
D_losses = []
iters = 0

plt.figure(figsize=(10,10))
print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(celeba_dataloader, 0):
        
        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 0, device=device).type(torch.float32)                
        output2 = netD(real_cpu).view(-1)              
        label.fill_(real_label)
        errD_real = criterion(output2, label)
        errD_real.backward()
        D_x = output2.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake2 = netG(noise)        
        output2 = netD(fake2.detach()).view(-1)           
        label.fill_(fake_label)        
        errD_fake2 = criterion(output2, label)
        errD_fake2.backward()
        D_G_z1 = output2.mean().item()
        errD = errD_real + errD_fake2
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label) 
        output2 = netD(fake2).view(-1)
        errG = criterion(output2, label)
        errG.backward()
        D_G_z2 = output2.mean().item()
        optimizerG.step()
        
        if i % 50 == 0:
            dsp_str = '[%d/%d][%d/%d]' % (epoch, num_epochs, i, len(celeba_dataloader))
            print('%s\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (dsp_str, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))       
            with torch.no_grad():
                fake2 = netG(fixed_noise_vmild).detach().cpu()
            fake2 = vutils.make_grid(fake2[:64], padding=2, normalize=True)
            plt.axis("off")
            plt.title("Generated fake Images")
            plt.imshow(np.transpose(vutils.make_grid(fake2, padding=2, normalize=True),(1,2,0)))
            plt.pause(1)

        G_losses.append(errG.item())
        D_losses.append(errD.item())
                    
        iters += 1

In [ ]:
#DO NOT RUN
from torchvision.utils import save_image

with torch.no_grad():
    final_fake_vmild = netG(fixed_noise_vmild).detach().cpu()
final_fake_grid_vmild = vutils.make_grid(final_fake_vmild, padding=2, normalize=True)


In [ ]:
#RUN
if not os.path.exists('results'):
    os.makedirs('results')

tensor_save_path_vmild = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakevmild.pt'
#torch.save(final_fake_grid_vmild, tensor_save_path_vmild)
fakevmild = torch.load(tensor_save_path_vmild)

In [ ]:
#DO NOT RUN
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Final Generated Fake Images")
plt.imshow(np.transpose(fakevmild, (1, 2, 0)))
plt.show()

## Mild

In [ ]:
#DO NOT RUN
mild = {}
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')
for i in range(2):
  for filename in os.listdir():
      if filename.endswith('.jpg'):
        mild[filename] = 'mild'

In [ ]:
#DO NOT RUN, Model already trained
import os
import zipfile 
import gdown
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

class CelebADataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.filenames = list(data_dict.keys())
        self.labels = list(data_dict.values())
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        label = self.labels[index]

        os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Mild_Demented')

        image = Image.open(filename).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        return image

image_size = 64
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

dataset = CelebADataset(mild, transform=transform)


batch_size = 1
num_workers = 0 if device.type == 'cuda' else 1
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=True)

In [ ]:
#DO NOT RUN

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

batch_size = 128
image_size = 64
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 5
lr = 0.0005
beta1 = 0.5
ngpu = 1

netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

criterion = nn.BCELoss()

fixed_noise_mild = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# DO NOT RUN, Training Loop

img_list = []
G_losses = []
D_losses = []
iters = 0

plt.figure(figsize=(10,10))
print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(celeba_dataloader, 0):

        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 0, device=device).type(torch.float32)                
        output3 = netD(real_cpu).view(-1)           
        label.fill_(real_label)
        errD_real = criterion(output3, label)
        errD_real.backward()
        D_x = output3.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake3 = netG(noise)        
        output3 = netD(fake3.detach()).view(-1)               
        label.fill_(fake_label)        
        errD_fake3 = criterion(output3, label)
        errD_fake3.backward()
        D_G_z1 = output3.mean().item()
        errD = errD_real + errD_fake3
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label)  
        output3 = netD(fake3).view(-1)
        errG = criterion(output3, label)
        errG.backward()
        D_G_z2 = output3.mean().item()
        optimizerG.step()
        
        if i % 50 == 0:
            dsp_str = '[%d/%d][%d/%d]' % (epoch, num_epochs, i, len(celeba_dataloader))
            print('%s\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (dsp_str, errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))       
            with torch.no_grad():
                fake3 = netG(fixed_noise_mild).detach().cpu()
            fake3 = vutils.make_grid(fake3[:64], padding=2, normalize=True)
            plt.axis("off")
            plt.title("Generated fake Images")
            plt.imshow(np.transpose(vutils.make_grid(fake3, padding=2, normalize=True),(1,2,0)))
            plt.pause(1)

        G_losses.append(errG.item())
        D_losses.append(errD.item())
                    
        iters += 1

In [ ]:
#DO NOT RUN
from torchvision.utils import save_image

with torch.no_grad():
    final_fake_mild = netG(fixed_noise_mild).detach().cpu()
final_fake_grid_mild = vutils.make_grid(final_fake_mild, padding=2, normalize=True)


In [ ]:
#RUN
if not os.path.exists('results'):
    os.makedirs('results')

tensor_save_path_mild = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakemild.pt'
#torch.save(final_fake_grid_mild, tensor_save_path_mild)
fakemild = torch.load(tensor_save_path_mild)

In [ ]:
#DO NOT RUN
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Final Generated Fake Images")
plt.imshow(np.transpose(fakemild, (1, 2, 0)))
plt.show()

## Test Model on Fakes

First Load the Models Trained in the 4 sections above

In [ ]:
#RUN
if not os.path.exists('results'):
    os.makedirs('results')

tensor_save_path_mod = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakemoderate.pt'
#torch.save(final_fake_grid_mod, tensor_save_path_mod)
fakemod = torch.load(tensor_save_path_mod)

tensor_save_path_non = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakenon.pt'
#torch.save(final_fake_grid_non, tensor_save_path_non)
fakenon = torch.load(tensor_save_path_non)

tensor_save_path_vmild = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakevmild.pt'
#torch.save(final_fake_grid_vmild, tensor_save_path_vmild)
fakevmild = torch.load(tensor_save_path_vmild)

tensor_save_path_mild = '/content/drive/MyDrive/Bio Final Project/Models/Fake Generation/fakemild.pt'
#torch.save(final_fake_grid_mild, tensor_save_path_mild)
fakemild = torch.load(tensor_save_path_mild)

In [ ]:
#RUN
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import torchvision.utils as vutils

top_left_mod = fakemod[0]
top_left_mod = top_left_mod[:128,:128]

top_left_mild = fakemild[0]
top_left_mild = top_left_mild[:64,:64]

top_left_vmild = fakevmild[0]
top_left_vmild = top_left_vmild[:64,:64]

top_left_non = fakenon[0]
top_left_non = top_left_non[:64,:64]

fig, axs = plt.subplots(2, 2, figsize=(8,8))

axs[0, 0].imshow(top_left_mod)
axs[0, 0].set_title('Moderate')

axs[0, 1].imshow(top_left_mild)
axs[0, 1].set_title('Mild')

axs[1, 0].imshow(top_left_vmild)
axs[1, 0].set_title('Very Mild')

axs[1, 1].imshow(top_left_non)
axs[1, 1].set_title('None')

plt.tight_layout()
plt.show()

testdic = {top_left_mod: "mod", top_left_mild: "mild", top_left_vmild: "vmild", top_left_non: "none"}


In [ ]:
image_size = 64
n_images_row = (fakemild.shape[2] - 2) // (image_size + 2) 
n_images_col = (fakemild.shape[1] - 2) // (image_size + 2)
mild_image_list = []

for row in range(n_images_row):
    for col in range(n_images_col):
        # Calculate the start and end positions of the image in the grid
        start_x = col * (image_size + 2) + 2
        end_x = start_x + image_size
        start_y = row * (image_size + 2) + 2
        end_y = start_y + image_size

        # Extract the image from the grid
        img = fakemild[:, start_y:end_y, start_x:end_x]
        mild_image_list.append(img)

image_size = 64
n_images_row = (fakevmild.shape[2] - 2) // (image_size + 2) 
n_images_col = (fakevmild.shape[1] - 2) // (image_size + 2)
vmild_image_list = []

for row in range(n_images_row):
    for col in range(n_images_col):
        # Calculate the start and end positions of the image in the grid
        start_x = col * (image_size + 2) + 2
        end_x = start_x + image_size
        start_y = row * (image_size + 2) + 2
        end_y = start_y + image_size

        # Extract the image from the grid
        img = fakevmild[:, start_y:end_y, start_x:end_x]
        vmild_image_list.append(img)

image_size = 128
n_images_row = (fakemod.shape[2] - 2) // (image_size + 2) 
n_images_col = (fakemod.shape[1] - 2) // (image_size + 2)
mod_image_list = []

for row in range(n_images_row):
    for col in range(n_images_col):
        # Calculate the start and end positions of the image in the grid
        start_x = col * (image_size + 2) + 2
        end_x = start_x + image_size
        start_y = row * (image_size + 2) + 2
        end_y = start_y + image_size

        # Extract the image from the grid
        img = fakemod[:, start_y:end_y, start_x:end_x]
        mod_image_list.append(img)

image_size = 64
n_images_row = (fakenon.shape[2] - 2) // (image_size + 2) 
n_images_col = (fakenon.shape[1] - 2) // (image_size + 2)
non_image_list = []

for row in range(n_images_row):
    for col in range(n_images_col):
        # Calculate the start and end positions of the image in the grid
        start_x = col * (image_size + 2) + 2
        end_x = start_x + image_size
        start_y = row * (image_size + 2) + 2
        end_y = start_y + image_size

        # Extract the image from the grid
        img = fakenon[:, start_y:end_y, start_x:end_x]
        non_image_list.append(img)


In [ ]:
#NEED TO LOAD ALEXNET FIRST BEFORE RUNNING THIS
#INSTUCTIONS TO DO THIS ABOVE IN THE "LOAD MODELS" SECTION
#"model" IS ALEXNET, BUT YOU CAN SWITCH IT TO A DIFFERENT ONE IF YOU WANT DIFFERENT RESULTS
from torchvision.transforms.functional import to_pil_image
import warnings
warnings.filterwarnings("ignore")

def classify_tensor(model, image_tensor, transform):
    image_pil = to_pil_image(image_tensor)

    image_rgb = image_pil.convert('RGB')

    if transform is not None:
        image_transformed = transform(image_rgb)

    image_transformed = image_transformed.unsqueeze(0)

    if torch.cuda.is_available():
        image_transformed = image_transformed.cuda()

    output = model(image_transformed)

    _, predicted_label = torch.max(output, 1)

    return predicted_label.item()

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#ALEXNET:
print("RESULTS USING ALEXNET:")
mildcount = 0
for img in mild_image_list:
  predicted_label = classify_tensor(model, img, transform=transform)
  if predicted_label == 0:
    mildcount += 1
print("Mild Accuracy = ", mildcount/len(mild_image_list)*100, "%" )

vmildcount = 0
for img in vmild_image_list:
  predicted_label = classify_tensor(model, img, transform=transform)
  if predicted_label == 1:
    vmildcount += 1
print("Very Mild Accuracy = ", vmildcount/len(vmild_image_list)*100, "%" )

modcount = 0
for img in mod_image_list:
  predicted_label = classify_tensor(model, img, transform=transform)
  if predicted_label == 2:
    modcount += 1
print("Moderate Accuracy = ", modcount/len(mod_image_list)*100, "%" )

noncount = 0
for img in non_image_list:
  predicted_label = classify_tensor(model, img, transform=transform)
  if predicted_label == 3:
    noncount += 1
print("Non Accuracy = ", noncount/len(non_image_list)*100, "%" )


# StyleGan3

We tried to incorporate StyleGan3 as a better method than what we have above, but it took too long and too many resources (even after buying colab+), but here is what we tried to do anyway

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

In [ ]:
import pickle
os.chdir('/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented')
data = []
for filename in os.listdir():
    if filename.endswith('.jpg'):
      data.append(filename)

with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('data.pkl', 'rb') as f:
    data = pickle.load(f)

os.makedirs('images', exist_ok=True)

for filename in data:
    image = Image.open(filename)
    image.save(os.path.join('images', filename))

In [ ]:
!apt-get update
!apt-get install ninja-build

In [ ]:
!python '/content/drive/MyDrive/Bio Final Project/stylegan3/train.py' --outdir='/content/drive/MyDrive/Bio Final Project/stylegan3/results' --cfg=stylegan3-t --data='/content/drive/MyDrive/Bio Final Project/Dataset/Very_Mild_Demented' --gpus=1 --batch=8 --gamma=8.2 --mirror=1

You can see above it took 50 minutes to run 0.16% of the data.

# References

Resnet: Lab 8 from class

VGG16 and Inception-V3: PyTorch Keras

Fine-Tuned Alexnet Model: https://github.com/wangyirui/AD_Prediction.git

Grad-Cam:
@misc{jacobgilpytorchcam,
  title={PyTorch library for CAM methods},
  author={Jacob Gildenblat and contributors},
  year={2021},
  publisher={GitHub},
  howpublished={\url{https://github.com/jacobgil/pytorch-grad-cam}},
}

DCGAN: Lab 10 from class

StyleGan3:
@inproceedings{Karras2021,
  author = {Tero Karras and Miika Aittala and Samuli Laine and Erik H\"ark\"onen and Janne Hellsten and Jaakko Lehtinen and Timo Aila},
  title = {Alias-Free Generative Adversarial Networks},
  booktitle = {Proc. NeurIPS},
  year = {2021}
}